In [18]:
import pandas as pd

Importo dei csv test e train in un unico dataframe 

In [19]:
df_test = pd.read_csv('data/test.csv')

df_train = pd.read_csv('data/train.csv')

df = pd.concat([df_test, df_train])

df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,NaN
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,NaN
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,NaN
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,NaN
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,NaN


Importo dizionario dei dati

In [20]:
df_dictionary = pd.read_csv('data/data_dictionary.csv')

row_count = len(df) 

display(df_dictionary)

,Variable,Definition,Key
0,survival,Survival,"0 = No, 1 = Yes"
1,pclass,Ticket class,"1 = 1st, 2 = 2nd, 3 = 3rd"
2,sex,Sex,NaN
3,Age,Age in years,
4,sibsp,# of siblings / spouses aboard the Titanic,NaN
5,parch,# of parents / children aboard the Titanic,NaN
6,ticket,Ticket number,NaN
7,fare,Passenger fare,NaN
8,cabin,Cabin number,NaN
9,embarked,Port of Embarkation,"C = Cherbourg, Q = Queenstown, S = Southampton"


-----
Esplorazione dei valori nulli colonna per colonna


- PassengerId

In [21]:
df_passenger_id = df['PassengerId']

passenger_id_null_counts = df_passenger_id.isnull().sum()

display(passenger_id_null_counts)

0

La colonna PassengerId non contiene valori nulli

- Pclass

In [22]:
df_Pclass = df['Pclass']

Pclass_null_counts = df_Pclass.isnull().sum()

display(Pclass_null_counts)

0

La colonna Pclass non contiene valori nulli

- Name

In [16]:
df_Name = df['Name']

Name_null_counts = df_Name.isnull().sum()

display(Name_null_counts)

0

- Sex

In [32]:
df_Sex = df['Sex']

Sex_null_counts = df_Sex.isnull().sum()

display(Sex_null_counts)

0

La colonna Sex non contiene valori nulli

- Age

In [34]:
df_Age= df['Age']

Age_null_counts = df_Age.isnull().sum()

display(Age_null_counts)

print(f"I dati relativi all'età mancano del {round(Age_null_counts/row_count*100,1)} %")

263

I dati relativi all'età mancano del 20.1 %


La colonna Age contiene 263 valori nulli

-- Capire se c'è un nesso o è semplicemente un data loss

- Parch

In [35]:
df_Parch= df['Parch']

Parch_null_counts = df_Parch.isnull().sum()

display(Parch_null_counts)

0

La colonna Parch non contiene valori nulli

- Fare

In [37]:
df_Fare= df['Fare']

Fare_null_counts = df_Fare.isnull().sum()

display(Fare_null_counts)

print(f'I dati relativi al costo dei biglietti mancano del {round(Fare_null_counts/row_count*100,1)} %')


1

I dati relativi al costo dei biglietti mancano del 0.1 %


La colonna Fare contiene 1 valore nullo

-- Capire se c'è un nesso o è semplicemente un data loss

- Cabin

In [31]:
df_Cabin = df['Cabin']

Cabin_null_counts = df_Cabin.isnull().sum()

display(Cabin_null_counts)

print(f'I dati relativi ai numeri delle cabine mancano del {round(Cabin_null_counts/row_count*100,1)} %')


1014

I dati relativi ai numeri delle cabine mancano del 77.5 %


La colonna Pclass contiene valori 1014 nulli

-- Capire se c'è un nesso o è semplicemente un data loss

- Embarked

In [39]:
df_Embarked= df['Embarked']

Embarked_null_counts = df_Embarked.isnull().sum()

display(Embarked_null_counts)

print(f'I dati relativi al porto di imbarco mancano del {round(Embarked_null_counts/row_count*100,1)} %')


2

I dati relativi al porto di imbarco mancano del 0.2 %


La colonna Embarked contiene valori 2 nulli

-- Capire se c'è un nesso o è semplicemente un data loss